In [1]:
import pandas as pd

from kiblib.utils.db import DbConn

In [2]:
db_conn = DbConn().create_engine()

# Prêts

In [5]:
#query = """SELECT * FROM statdb.stat_issues si WHERE YEAR(si.issuedate)"""
#statdb_issues = pd.read_sql(query,db_conn)

statdb_issues = pd.read_csv("/home/kibini/statdb_issues.tsv.gz", sep="\t")

/tmp/ipykernel_21205/2638905257.py:4: DtypeWarning: Columns (12,20,23) have mixed types. Specify dtype option on import or set low_memory=False.
  statdb_issues = pd.read_csv("/home/kibini/statdb_issues.tsv.gz", sep="\t")


In [6]:
statdb_issues['issuedate'] = pd.to_datetime(statdb_issues['issuedate'])

In [9]:
len(statdb_issues)

343864

In [8]:
statdb_issues = statdb_issues[statdb_issues['issuedate'].dt.year == 2023]

In [10]:
statdb_issues = statdb_issues[~statdb_issues['itemnumber'].isna()]
statdb_issues = statdb_issues[~statdb_issues['issuedate'].isna()]
statdb_issues['itemnumber'] = statdb_issues['itemnumber'].astype(int)
statdb_issues = statdb_issues[~statdb_issues['itemnumber'].isna()]
statdb_issues['itemnumber'] = statdb_issues['itemnumber'].astype(int)
statdb_issues['cle'] =  statdb_issues['issuedate'].astype(str) + "-" + statdb_issues['itemnumber'].astype(str)

In [11]:
#query = """SELECT * FROM koha_prod.issues si"""
#df1 = pd.read_sql(query,db_conn)
df1 = pd.read_csv("/home/kibini/koha_prod_issues.tsv.gz", sep="\t")

#query = """SELECT * FROM koha_prod.old_issues si3"""
#df2 = pd.read_sql(query,db_conn)

df2 = pd.read_csv("/home/kibini/koha_prod_old_issues.tsv.gz", sep="\t")
koha_issues = pd.concat([df1, df2])
koha_issues['issuedate'] = pd.to_datetime(koha_issues['issuedate'])
koha_issues = koha_issues[koha_issues['issuedate'].dt.year == 2023]

koha_issues = koha_issues[~koha_issues['itemnumber'].isna()]
koha_issues = koha_issues[~koha_issues['issuedate'].isna()]
koha_issues['itemnumber'] = koha_issues['itemnumber'].astype(int)
koha_issues = koha_issues[~koha_issues['itemnumber'].isna()]
koha_issues['itemnumber'] = koha_issues['itemnumber'].astype(int)
koha_issues['cle'] =  koha_issues['issuedate'].astype(str) + "-" + koha_issues['itemnumber'].astype(str)

/tmp/ipykernel_21205/3597028274.py:8: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("/home/kibini/koha_prod_old_issues.tsv.gz", sep="\t")


In [12]:
statdb_issues['import'] = True
print(len(statdb_issues))
print(len(koha_issues))
print(f"Manque : {len(koha_issues) - len(statdb_issues)}")

343863
359430
Manque : 15567


In [28]:
missing = koha_issues.merge(statdb_issues[['cle', 'import']], how='outer', on=['cle'])
missing = missing[missing['import'].isna()]
len(missing)

22089

In [31]:
missing['cle'].to_csv("missing.csv", index=False)